В данном примере будет рассмотрен базовый вейвлет анализ стационарных и нестационарных сигналов. Будут рассмотрены некоторые различия БПФ преобразования (в том числе оконного преобразования Фурье) от вейвлет преобразования. Преобразование Фурье хорошо справляется с сигналами в которых частоты присутствуют одновременно и их присутствие не обусловлено зависимостью от времени (стационарные сигналы), однако в силу определённых ограничений плохо справляется с нестационарными сигналами. Вейвлет анализ помогает решить эту проблему. В частности будут рассмотрен особый тип представления вейвлет преобразованя - скалограмма.

Данный пример может быть загружен по адресу: https://github.com/sven4500/num-analysis

In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot
%run ../lab-3/lab-3-spectrogram.ipynb

In [ ]:
n = 2000
n_4 = n // 4
t = np.linspace(0., 2 * np.pi, n, endpoint=False)

y_0 = 0.
y_1 = 3. * np.cos(5 * t)
y_2 = 7. * np.cos(13 * t)
y_3 = 5. * np.cos(19 * t)
y_4 = 11. * np.cos(31 * t)

y_static = y_0 + y_1 + y_2 + y_3 + y_4

Сигнал состоит из 4 частот. На этом сигнале все частоты присутствуют одновременно.

In [ ]:
matplotlib.pyplot.plot(t, y_static)

После БПФ видим эти частоты в частотной области сигнала.

In [ ]:
df_static = np.fft.fft(y_static)
matplotlib.pyplot.plot(np.abs(df_static)[:40], '--')

Теперь представим сигнал в котором наличие частот зависит от времени. Ниже представлен сигнал в котором писутствуют те же частоты однако они присутствуют в сигнале не одновременно, а в разное время по одной частоте на каждом отрезке времени.

In [ ]:
y_dynamic = np.concatenate([y_1[0*n_4:1*n_4], y_2[1*n_4:2*n_4], y_3[2*n_4:3*n_4], y_4[3*n_4:4*n_4]])
matplotlib.pyplot.plot(y_dynamic)
assert len(y_dynamic) == len(y_static)

В частотной области этого сигнала невозможно явно оперделить какие частоты присутствуют в сигнале. На рисунке в принципе видны пики на соответствующих частотах, однако картина получается размытой.

In [ ]:
df_dynamic = np.fft.fft(y_dynamic)
matplotlib.pyplot.plot(np.abs(df_dynamic)[:40], '--')

Можно посмотреть на частотный спектр обих сигналов одновременно с нормализацией. Видим что частоты для стационарного и нестационарного сигналов в целом соответствуют друг другу, однако в случае нестационарного сигнала спектр не чёткий.

In [ ]:
df_static_max = np.amax(np.abs(df_static))
df_dynamic_max = np.amax(np.abs(df_dynamic))

matplotlib.pyplot.plot(np.abs(df_static)[:40] / df_static_max, '--')
matplotlib.pyplot.plot(np.abs(df_dynamic)[:40] / df_dynamic_max, '--')

Для анализа нестационарного сигнала можно использовать оконное преобразование Фурье. Результат оконного преобразования Фурье это двумерная картинка в которой во горизонтальной оси отложено время, а по вертикальной оси частоты. Цветом кодируется интенсивность или амплитуда этой частоты. Таким образом оконное преобразование Фурье показывает эволюцию частот во времени. На картинках ниже представлено оконное преобразование Фурье.

In [ ]:
s_1 = spectrogram(y_dynamic, 2048, 1.0)
s_2 = spectrogram(y_dynamic, 2048, 500.0)

fig, ax = matplotlib.pyplot.subplots(2,1)

#ax[0].set_title('Широкое окно - неопределённость по времени')
ax[0].matshow(np.abs(s_1).T[:100,:], aspect='auto')

#ax[1].set_title('Узкое окно - неопределённость по частотам')
ax[1].matshow(np.abs(s_2).T[:100,:], aspect='auto')

Оконное преобразование Фурье имеет недостаток который характеризуется неопределённостью времени и частоты. Чем-то напоминает неопределённость Гейзенберга. Чем точнее на спектрограмме время, тем расплывчатее частоты (нижний рисунок) и наоборот чем точнее частоты тем менее определённое время (верхний рисунок).

Для того чтобы анализировать такие нестационарные сигналы можно воспользоваться вейвлет анализом. Библиотека __scipy__ имеет средства для работы с вейвлетами.

In [ ]:
import scipy
from scipy import signal

Вейвлет это особая конечная во времени функция которая используется как базисная функция в терминах которой раскладывается сигнал. Преобразование Фурье в свою очередь раскладывает сигнал на гармонические функции синуса или косинуса. Ниже представлен вейвлет Рикера также известный как вейвлет "мексиканская шляпа" для различных коэффициентов растяжения.

In [ ]:
n_wt = 150

mexh_1 = scipy.signal.ricker(n_wt, 1.)
mexh_5 = scipy.signal.ricker(n_wt, 5.)
mexh_20 = scipy.signal.ricker(n_wt, 20.)

matplotlib.pyplot.plot(mexh_1)
matplotlib.pyplot.plot(mexh_5)
matplotlib.pyplot.plot(mexh_20)

Вейвлет преобразование как и преобразование Фурье может быть как прямым, так и обратным. Прямое вейвлет преобразование определяется как свъёртка исходного сигнала с заданным вейвлетом. На картинке ниже представлено прямое вейвлет преобразование для вейвлета с коэффициентом растяжения 5.

In [ ]:
dwt = scipy.signal.convolve(y_dynamic, scipy.signal.ricker(n, 5.), mode='same')
matplotlib.pyplot.plot(dwt)

В составе библиотеки __scipy__ имеется функция __cwt__ которая может посчитать пямое вейвлет преобразование сразу для серии вейвлетов (т.е. для вейвлетов с различными коэффициентами растяжения). Ниже представлен результат вейвлет преобразования нестационарного сигнала для серии коэффициентов растяжения вейвлета "мексиканская шляпа".

In [ ]:
widths = np.arange(1, 80)
scalogram = scipy.signal.cwt(y_dynamic, scipy.signal.ricker, widths) # <- ricker == mexican hat

for i in scalogram[:5]:
    matplotlib.pyplot.plot(i)

matplotlib.pyplot.ylim(top=5, bottom=-5)

Народобие спектрограммы которая получается с помощью оконного преобразования Фурье, вейвлет преобразование также можно рассмотреть на двумерной плоскости если расположить результат вейвлет преобразования так как это делается для построения спектрограммы. Такой рисунок называется __скалограммой__ (англ. scale растяжение). На скалограмме по горизонтале отложено время (точно также как на спектрограмме) однако по вертикале отложен коэффициент растяжения вейвлета. На картинке ниже видно явно выделенные временные домены в которых присутствует только один вейвлет.

In [ ]:
matplotlib.pyplot.imshow(scalogram, aspect='auto')

Существует ещё один модуль __pywt__ (Python Wavelets) который используется для работы с вейвлетами.

In [ ]:
import pywt

Этот модуль разделяет дискретное и непрерывное вейвлет преобразование. Доступные в этом пакете дискретные и непрерывные вейвлеты можно получить вызвав одну из следующих команду ниже.

In [ ]:
print(pywt.wavelist(kind='discrete'), pywt.wavelist(kind='continuous'), sep='\n\n')

Непрерывное вейвлет преобразование можно выполнить вызвав метод __cwt__ (англ. Continuous Wavelet Transform). Результат работы этой функции аналогичен вызову метода из модуля __scipy__. Скалограмма также идентичная скалограмме полученной средствами библиотеки __scipy__.

In [ ]:
cwt, freq = pywt.cwt(y_dynamic, np.arange(1, 80), wavelet='mexh')
matplotlib.pyplot.imshow(cwt, aspect='auto')

Функция __dwt__ производит дискретное вейвлет преобразование сигнала. Название дискретное __dwt__ и непрерывное __cwt__ вейвлет преобразование в контексте компьютерных вычислений может сбить с толку. Оба типа преобразований выполняются на компрьютере и оба типа преобразований являются "дискретными". __dwt__ было разработано специально чтобы повысить скорость работы вейвлет анализа. Иногда __dwt__ называют FWT (_англ._ Fast Wavelet Transform) по аналогии с FFT (_англ._ Fast Fourier Transform). В целом __dwt__ отличается от __cwt__ по нескольким пунктам:

1. __dwt__ предполагает что не все данные представляют интерес для обработки поэтому после определённых допущений результат преобразования сокращается в 2 раза (так называемый каскадный алгоритм);

2. __dwt__ по мимо вейвлета использует ещё одну функцию которая называется масштабирующей функцией;

3. __dwt__ реализует свёртку сигнала немного иначе.

В целом __dwt__ и __cwt__ носят одинаковую смысловую нагрузку. Таким образом, там где например необходима высокая сеорость обработки вейвлетов или алгоритмы передачи данных используют вейвлет анализ можно использовать быстрый __dwt__. Там где задача предполагает исследовательскую работу нужно использовать __cwt__.

In [ ]:
_, dwt = pywt.dwt(y_dynamic, 'db2') # <- вейвлет Дебоши
matplotlib.pyplot.plot(dwt)

Можно провести серию преобразований __dwt__ которе делается с помощью функции __wavedec__. Из графика видно что размер __dwt__ преобразование на каждом последующем уровне менше в 2 раза.

In [ ]:
#dwt = pywt.dwtn(y_dynamic, 'db2')
#swt = pywt.swt(y_dynamic, 'db2')
wavedec = pywt.wavedec(y_dynamic, 'db2', level=4)
for i in wavedec[1:]:
    matplotlib.pyplot.plot(i)
    print(len(i))

Модуль __scaleogram__ ещё один отдельный модуль для работы с вейвлетами однако его основная функция это построение скалограмм.

In [ ]:
import scaleogram as scg

Метод __set_default_wavelet__ позволяет задат вейвлет по умолчанию чтобы явно не указывать тип вейвлета при всех дальнейших вызовах функций.

In [ ]:
#scg.set_default_wavelet('cmor1-1.5')
#scg.set_default_wavelet('cgau5')
#scg.set_default_wavelet('cgau1')
#scg.set_default_wavelet('shan0.5-2') # <- вейвлет Шеннона
#scg.set_default_wavelet('mexh') # <- вейвлет Рикера (мексиканская шляпа)

Скалограмма построенная средствами библиотеки __scaleogram__ идентична уже ранее построеной скалеограмме средствами библиотеки __scipy__ или __pywt__.

In [ ]:
scales = scg.periods2scales(np.arange(1, 50))
#scales = scg.periods2scales([5, 13, 19, 31])
scg.cws(y_dynamic, scales=scales, wavelet='mexh') # figsize=(6.9,2.3)

Тот же, но стационарный сигнал показан на картинке ниже. На картинке видно что в каждый момент времени сигнал состоит из нескольких вейвлетов одновременно в отличие от нестационарного сигнала.

In [ ]:
scales = scg.periods2scales(np.arange(1, 300))
scg.cws(y_static, scales=scales, wavelet='cmor1-1.5')

С помощью вейвлет анализа можно посмотреть шум.

In [ ]:
noise = np.random.normal(size=200)
matplotlib.pyplot.plot(noise)

scales = scg.periods2scales(np.arange(1, 100))
scg.cws(noise, scales=scales)

- [ ] покажи применение вейвлетов для изображений
- [ ] покажи как с помощью вейвлетов можно провести анализ и синтез. например для очистки изображения от шума.

Итоги

В данном примере были рассмотрены три библиотеки: __scipy__, __pywt__ и __scalogram__. Каждая библиотека имеет свои особенности. Непрерывное вейвлет преобразование можно пострить с помощью как __scipy__, так и __pywt__. Однако в библиотеке __scipy__ нехватает разнообразия вейвлет функций по сравнению со специализированной библиотекой __pywt__. Библиотека __scaleogram__ рассчитана для быстрого построения скалограм. В целом достаточно только модуля __scipy__ для самых базовых операций в вейвлет анализе. В остальном стоит воспользоваться модулем __pywt__. Если требуется представление результата на скалограмме, то стоит воспользоваться модулем __scaleogram__ что рисования красивых графиков.

Источники:

[1] A guide for using the Wavelet Transform in Machine Learning // http://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/

[2] Time series features extraction using Fourier and Wavelet transforms on ECG data // https://blog.octo.com/en/time-series-features-extraction-using-fourier-and-wavelet-transforms-on-ecg-data/

[3] A gentle introduction to wavelet for data analysis // https://www.kaggle.com/asauve/a-gentle-introduction-to-wavelet-for-data-analysis/notebook?scriptVersionId=12579739#Introduction

[4] Multiple Time Series Classification by Using Continuous Wavelet Transformation // https://towardsdatascience.com/multiple-time-series-classification-by-using-continuous-wavelet-transformation-d29df97c0442

[5] Introduction to Wavelets in Image Processing // http://inside.mines.edu/~whoff/courses/EENG510/lectures/24-Wavelets.pdf

[6] Using continuous verses discrete wavelet transform in digital applications // https://dsp.stackexchange.com/questions/8009/using-continuous-verses-discrete-wavelet-transform-in-digital-applications

[7] What is the difference between the Continuous and Discrete wavelet transform? // https://www.quora.com/What-is-the-difference-between-the-Continuous-and-Discrete-wavelet-transform